# Benchmark PyRCN

This notebook accompanies the publication "PyRCN: A Toolbox for Exploration and Application of Reservoir Computing Networks" and serves as a benchmark test for PyRCN and other libraries.

Technical analysis -> Chart Analyse

In [1]:
import pandas as pd
import itertools
from collections import OrderedDict

from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from joblib import dump, load

from pyrcn.extreme_learning_machine import ELMRegressor
from pyrcn.echo_state_network import ESNRegressor
from pyrcn.model_selection import SequentialSearchCV
from skelm import ELMRegressor as SkELMRegressor
import seaborn as sns
import matplotlib.pyplot as plt

from src.preprocessing import ts2super, compute_average_volatility
from src.model_selection import PredefinedTrainValidationTestSplit
from src.adapter import PyESN


%matplotlib inline
sns.set_theme()

## Prepare all datasets

We load the datasets as pandas dataframes

In [2]:
datasets = OrderedDict({"CAT": 0, "EBAY": 1, "MSFT": 2})
data = [None] * len(datasets)

for dataset, k in datasets.items():
    data[k] = pd.read_csv(f"./data/{dataset}.csv")["t0"].to_frame()

Next, we create feature extraction pipelines.

In [3]:
day_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 1})
week_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 5})
month_volatility_transformer = FunctionTransformer(
    func=compute_average_volatility, kw_args={"window_length": 22})
har_features = FeatureUnion(
    transformer_list=[("day", day_volatility_transformer),
                      ("week", week_volatility_transformer),
                      ("month", month_volatility_transformer)])
har_pipeline = Pipeline(
    steps=[("har_features", har_features),
           ("scaler", MinMaxScaler()),
           ("lstsq", TransformedTargetRegressor(transformer=MinMaxScaler()))])

We define a forecasting horizon for which we want to optimize and analyze models

In [70]:
H=21  # pre-trained models available for 1, 5, 21

# ts2super simply returns a dataframe containing the original and shifted by H time series.
df = pd.concat([ts2super(d, 0, H) for d in data])
X = df.iloc[:, 0].values.reshape(-1, 1)
y = df.iloc[:, -1].values.reshape(-1, 1)

In [71]:
df_input = df.iloc[:, 0].to_frame()
df_input["t5"] = df_input.t0.rolling(window=5, min_periods=1).mean()
df_input["t22"] = df_input.t0.rolling(window=22, min_periods=1).mean()
df_input

,t0,t5,t22
0,0.000223,0.000223,0.000223
1,0.000121,0.000172,0.000172
2,0.000124,0.000156,0.000156
3,0.000268,0.000184,0.000184
4,0.000832,0.000314,0.000314
...,...,...,...
2719,0.000048,0.000080,0.000162
2720,0.000027,0.000068,0.000150
2721,0.000037,0.000057,0.000138
2722,0.000070,0.000054,0.000129


In [72]:
df_input_extended = pd.concat([pd.DataFrame(data={"t0": [0], "t5": [0], "t22": [0]}), df_input], ignore_index=False).reset_index(drop=True)
df_input_extended["t0"] = df_input_extended["t0"].shift(periods=-1)
df_input_extended = df_input_extended.dropna()

In [73]:
df_input_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8172 entries, 0 to 8171
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   t0      8172 non-null   float64
 1   t5      8172 non-null   float64
 2   t22     8172 non-null   float64
dtypes: float64(3)
memory usage: 255.4 KB


Provide a custom train test split for this task, and all scores that we aim to evaluate.

In [74]:
test_fold = [[k] * len(ts2super(d, 0, H)) for k, d in enumerate(data)]
test_fold = list(itertools.chain.from_iterable(test_fold))

ps = PredefinedTrainValidationTestSplit(test_fold=test_fold, validation=True)
ps_test = PredefinedTrainValidationTestSplit(test_fold=test_fold, validation=False)

scoring = {"MSE": "neg_mean_squared_error", "RMSE": "neg_root_mean_squared_error", "R2": "r2"}

## HAR

The HAR (Heterogeneous AutoRegressive) features are, as the name says, autoregressive features. Here, temporal context is considered by smoothing a time series with moving averages of the last week (5 days) and month (22 days). The smoothed variants are finally added to the original time series. Thus, HAR has three features.

TODO: HAR-Feature extraction outside of the pipeline

In [75]:
har_pipeline = Pipeline(
    steps=[# ("har_features", har_features),
           ("scaler", MinMaxScaler()),
           ("lstsq", TransformedTargetRegressor(transformer=MinMaxScaler()))])

In [76]:
try:
    search = load(f'./results/har_grid_h{H}_modified.joblib')
except FileNotFoundError:
    search = GridSearchCV(estimator=har_pipeline, param_grid={}, cv=ps_test,
                          scoring=scoring, refit="R2", return_train_score=True, verbose=10).fit(df_input_extended, y)
    dump(search, f'./results/har_grid_h{H}_modified.joblib')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3; 1/1] START ............................................................
[CV 1/3; 1/1] END  MSE: (train=-0.000, test=-0.000) R2: (train=0.292, test=0.305) RMSE: (train=-0.000, test=-0.000) total time=   0.0s
[CV 2/3; 1/1] START ............................................................
[CV 2/3; 1/1] END  MSE: (train=-0.000, test=-0.000) R2: (train=0.311, test=0.241) RMSE: (train=-0.000, test=-0.000) total time=   0.0s
[CV 3/3; 1/1] START ............................................................
[CV 3/3; 1/1] END  MSE: (train=-0.000, test=-0.000) R2: (train=0.290, test=0.243) RMSE: (train=-0.000, test=-0.000) total time=   0.0s


In [77]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search.cv_results_['mean_train_R2']}\t{search.cv_results_['mean_test_R2']}\t"
      f"{search.cv_results_['mean_train_MSE']}\t{search.cv_results_['mean_test_MSE']}\t"
      f"{search.cv_results_['mean_fit_time']*1e3}\t{search.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.29726028]	[0.2628909]	[-1.17106478e-07]	[-1.18731243e-07]	[13.36868604]	[5.51358859]


## PyRCN ESN

Next, we optimize ESNs from PyRCN using only the original time series as features.

In [78]:
try:
    search = load(f'./results/pyrcn_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'identity',
        'bias_scaling': 0.0, 'spectral_radius': 0.0, 'leakage': 1.0, 'k_rec': 10,
        'reservoir_activation': 'tanh', 'bidirectional': False, 'alpha': 1e-5, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=ESNRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__leakage': uniform(1e-5, 1e0)}
    step3_params = {'esn__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step4_params = {'esn__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step4 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3),
        ('step4', RandomizedSearchCV, step4_params, kwargs_step4)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyrcn_seq_esn_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'esn__regressor__input_scaling': 0.20967378215835974, 'esn__regressor__spectral_radius': 1.0284688768272232}, 'step2': {'esn__regressor__leakage': 0.31172107608941096}, 'step3': {'esn__regressor__bias_scaling': 0.06175348288740734}, 'step4': {'esn__regressor__alpha': 0.0039054412752107916, 'esn__regressor__hidden_layer_size': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [79]:
try:
    search_test = load(f'./results/pyrcn_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyrcn_seq_esn_h{H}_test.joblib')

In [80]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.36171414]	[0.31596711]	[-1.06168016e-07]	[-1.09820392e-07]	[403.71926626]	[150.03554026]


Next, we optimize ESNs from PyRCN using HAR as features.

In [81]:
try:
    search = load(f'./results/pyrcn_har_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'identity',
        'bias_scaling': 0.0, 'spectral_radius': 0.0, 'leakage': 1.0, 'k_rec': 10,
        'reservoir_activation': 'tanh', 'bidirectional': False, 'alpha': 1e-5, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=ESNRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__leakage': uniform(1e-5, 1e0)}
    step3_params = {'esn__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step4_params = {'esn__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step4 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3),
        ('step4', RandomizedSearchCV, step4_params, kwargs_step4)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyrcn_har_seq_esn_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'esn__regressor__input_scaling': 0.1548948720912231, 'esn__regressor__spectral_radius': 0.978905520555126}, 'step2': {'esn__regressor__leakage': 0.30425224295953773}, 'step3': {'esn__regressor__bias_scaling': 0.46798356100860794}, 'step4': {'esn__regressor__alpha': 0.0039054412752107916, 'esn__regressor__hidden_layer_size': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [82]:
try:
    search_test = load(f'./results/pyrcn_har_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyrcn_har_seq_esn_h{H}_test.joblib')

In [83]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.35745346]	[0.30969009]	[-1.0706213e-07]	[-1.11069823e-07]	[416.5306886]	[174.58772659]


## PyRCN ELM

Next, we optimize ELMs from PyRCN using only the original time series as features.

In [84]:
try:
    search = load(f'./results/pyrcn_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'tanh',
        'bias_scaling': 0.0, 'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("'random_state': 42", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'esn__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyrcn_seq_elm_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'elm__regressor__input_scaling': 0.7419939418114051}, 'step2': {'elm__regressor__bias_scaling': 0.06175348288740734}, 'step3': {'elm__regressor__alpha': 0.0039054412752107916, 'elm__regressor__hidden_layer_size': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [85]:
try:
    search_test = load(f'./results/pyrcn_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyrcn_seq_elm_h{H}_test.joblib')

In [86]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.25572784]	[0.21816533]	[-1.23882321e-07]	[-1.26022426e-07]	[33.09901555]	[10.55185]


Next, we optimize ELMs from PyRCN using HAR as features.

In [87]:
try:
    search = load(f'./results/pyrcn_har_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'tanh',
        'bias_scaling': 0.0, 'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("'random_state': 42", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'elm__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyrcn_har_seq_elm_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'elm__regressor__input_scaling': 0.10767211400638386}, 'step2': {'elm__regressor__bias_scaling': 0.06175348288740734}, 'step3': {'elm__regressor__alpha': 0.0039054412752107916, 'elm__regressor__hidden_layer_size': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [88]:
try:
    search_test = load(f'./results/pyrcn_har_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyrcn_har_seq_elm_h{H}_test.joblib')

In [89]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.3056951]	[0.27217819]	[-1.15662543e-07]	[-1.1714314e-07]	[31.37811025]	[10.69545746]


## HP-ELM

Next, we optimize ELMs from HP-ELM using only the original time series as features.

In [90]:
try:
    search = load(f'./results/skelm_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'tanh',
        'bias_scaling': 0.0, 'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("'random_state': 42", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'elm__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(elm_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/skelm_seq_elm_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'elm__regressor__alpha': 1.3992515562068698e-05, 'elm__regressor__n_neurons': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [91]:
try:
    search_test = load(f'./results/skelm_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/skelm_seq_elm_h{H}_test.joblib')

In [92]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.23639608]	[0.19482808]	[-1.27086115e-07]	[-1.29155482e-07]	[18.56168111]	[8.52592786]


Next, we optimize ELMs from HP-ELM using HAR as features.

In [93]:
try:
    search = load(f'./results/skelm_har_seq_elm_h{H}.joblib')
except FileNotFoundError:
    initial_elm_params = {
        'hidden_layer_size': 50, 'k_in': 1, 'input_scaling': 0.4, 'input_activation': 'tanh',
        'bias_scaling': 0.0, 'alpha': 1e-5, 'random_state': 42}
    elm_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("'random_state': 42", TransformedTargetRegressor(
                                       regressor=ELMRegressor(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'elm__regressor__input_scaling': uniform(loc=1e-2, scale=1)}
    step2_params = {'elm__regressor__bias_scaling': uniform(loc=0, scale=3)}
    step3_params = {'elm__regressor__hidden_layer_size': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'elm__regressor__alpha': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step3 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2),
        ('step3', RandomizedSearchCV, step3_params, kwargs_step3)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/skelm_har_seq_elm_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'elm__regressor__alpha': 1.453542417265149, 'elm__regressor__n_neurons': 50}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [94]:
try:
    search_test = load(f'./results/pyrcn_har_seq_elm_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyrcn_har_seq_elm_h{H}_test.joblib')

In [95]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.3056951]	[0.27217819]	[-1.15662543e-07]	[-1.1714314e-07]	[31.37811025]	[10.69545746]


## PyESN ESN

Next, we optimize ESNs from PyESN using only the original time series as features.

In [97]:
try:
    search = load(f'./results/pyesn_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'n_inputs': 1, 'n_outputs': 1, 'n_reservoir': 50, 'spectral_radius': 0.0, 'sparsity': 0.5,
        'noise': 1e-3, 'input_shift': 0, 'input_scaling': 0.4, 'teacher_forcing': True,
        'teacher_scaling': 1.,'teacher_shift': 0, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=PyESN(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__n_reservoir': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__noise': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyesn_seq_esn_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'esn__regressor__input_scaling': 0.1295942459383017, 'esn__regressor__spectral_radius': 1.42648957444599}, 'step2': {'esn__regressor__n_reservoir': 50, 'esn__regressor__noise': 1.246802066209201e-05}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [98]:
try:
    search_test = load(f'./results/pyesn_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyesn_seq_esn_h{H}_test.joblib')

In [99]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.28610938]	[0.26937921]	[-1.19052607e-07]	[-1.17368253e-07]	[107.71107674]	[53.97764842]


Next, we optimize ESNs from PyESN using HAR as features.

In [96]:
try:
    search = load(f'./results/pyesn_har_seq_esn_h{H}.joblib')
except FileNotFoundError:
    initial_esn_params = {
        'n_inputs': 1, 'n_outputs': 1, 'n_reservoir': 50, 'spectral_radius': 0.0, 'sparsity': 0.5,
        'noise': 1e-3, 'input_shift': 0, 'input_scaling': 0.4, 'teacher_forcing': True,
        'teacher_scaling': 1.,'teacher_shift': 0, 'random_state': 42}
    esn_pipeline = Pipeline(steps=[("har_features", har_features),
                                   ("scaler", MinMaxScaler()),
                                   ("esn", TransformedTargetRegressor(
                                       regressor=PyESN(**initial_esn_params),
                                       transformer=MinMaxScaler()))
                                  ])
    # Run model selection
    step1_params = {'esn__regressor__input_scaling': uniform(loc=1e-2, scale=1),
                    'esn__regressor__spectral_radius': uniform(loc=0, scale=2)}
    step2_params = {'esn__regressor__n_reservoir': [50, 100, 200, 400, 800, 1600, 3200, 6400],
                    'esn__regressor__noise': loguniform(1e-5, 1e1)}
    
    kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}
    kwargs_step2 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring,
                    "refit": "R2", "cv": ps, "return_train_score": True}

    searches = [
        ('step1', RandomizedSearchCV, step1_params, kwargs_step1),
        ('step2', RandomizedSearchCV, step2_params, kwargs_step2)]
    search = SequentialSearchCV(esn_pipeline, searches=searches).fit(X, y)
    dump(search, f'./results/pyesn_har_seq_esn_h{H}.joblib')
print(search.all_best_params_)

{'step1': {'esn__regressor__input_scaling': 0.24763754399239968, 'esn__regressor__spectral_radius': 1.4564326972237192}, 'step2': {'esn__regressor__n_reservoir': 50, 'esn__regressor__noise': 9.692658471430319}}


We do an additional GridSearchCV without parameters, because the object returns a mass of useful information.

If we do it without parameters, simply the estimator will be fit on the train test splits defined in our cv.

In [68]:
try:
    search_test = load(f'./results/pyesn_har_seq_esn_h{H}_test.joblib')
except FileNotFoundError:
    search_test = GridSearchCV(estimator=clone(search.best_estimator_), param_grid={}, cv=ps_test,
                               scoring=scoring, refit="R2", return_train_score=True).fit(X, y)
    dump(search_test, f'./results/pyesn_har_seq_esn_h{H}_test.joblib')

In [69]:
print(f"R^2 train\tR^2 test\tMSE train\tMSE test\tFit time in ms\tScore time in ms")
print(f"{search_test.cv_results_['mean_train_R2']}\t{search_test.cv_results_['mean_test_R2']}\t"
      f"{search_test.cv_results_['mean_train_MSE']}\t{search_test.cv_results_['mean_test_MSE']}\t"
      f"{search_test.cv_results_['mean_fit_time']*1e3}\t{search_test.cv_results_['mean_score_time']*1e3}")

R^2 train	R^2 test	MSE train	MSE test	Fit time in ms	Score time in ms
[0.55682322]	[0.51334485]	[-7.29200176e-08]	[-7.87982402e-08]	[104.21069463]	[51.16319656]
